## RNN Overview

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" alt="nn" style="width: 600px;"/>

References:
- [Long Short Term Memory](http://deeplearning.cs.cmu.edu/pdfs/Hochreiter97_lstm.pdf), Sepp Hochreiter & Jurgen Schmidhuber, Neural Computation 9(8): 1735-1780, 1997.

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

To classify images using a recurrent neural network, we consider every image row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then handle 28 sequences of 28 timesteps for every sample.


In [0]:
# !pip install tensorflow-gpu
!pip install tensorflow==1.3.0

     |████████████████████████████████| 43.1MB 1.6MB/s 
     |████████████████████████████████| 1.6MB 37.1MB/s 
     |████████████████████████████████| 890kB 46.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
ERROR: fastai 0.7.0 has requirement torch<0.4, but you'll have torch 1.1.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.3.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.0
    Uninstalling bleach-3.1.0:
      Successfully uninstalled bleach-3.1.0
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


In [0]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, SimpleRNN
from tensorflow.keras.models import save_model

In [0]:
import os

In [0]:
mnist = tf.keras.datasets.mnist  # mnist is a dataset of 28x28 images of handwritten digits and their labels
(x_train, y_train),(x_test, y_test) = mnist.load_data()  # unpacks images to x_train/x_test and labels to y_train/y_test

x_train = x_train/255.0
x_test = x_test/255.0

print(x_train.shape)
print(x_train[0].shape)
model = Sequential()
model.add(LSTM(128, input_shape=(x_train.shape[1:]), activation='relu', return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(128, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(10, activation='softmax'))

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(28, 28)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)

model.fit(x_train,
          y_train,
          epochs=1)



60000/60000 [==============================] - 429s 7ms/sample - loss: 0.6265 - acc: 0.7908


In [0]:
model.evaluate(x_test,y_test)

10000/10000 [==============================] - 12s 1ms/sample - loss: 0.1522 - acc: 0.9548


[0.15221235538348554, 0.9548]

In [0]:
y_hat = model.predict(x_train)

In [0]:
model.save('/content/rnn_cell.h5')

In [0]:
model = tf.keras.models.load_model('/content/rnn_cell.h5')
print(model.outputs)


[<tf.Tensor 'dense_1_1/Softmax:0' shape=(?, 10) dtype=float32>]


In [0]:
# from keras import backend as K
from tensorflow.keras import backend as K


# # This line must be executed before loading Keras model.
K.set_learning_phase(0)


In [0]:
# from show_graph import show_graph
# from keras import backend as K
# import tensorflow as tf
sess = K.get_session()
graph_def = sess.graph.as_graph_def()
# graph_def
# show_graph(graph_def)


In [0]:
# from tensorflow.keras import backend as K
# import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 90 variables.
INFO:tensorflow:Converted 90 variables to const ops.


In [0]:
os.makedirs('/content/model')

In [0]:
tf.train.write_graph(frozen_graph, "/content/saved", "rnn_cell.pb", as_text=False)

NameError: ignored

In [0]:
! tflite_convert \
 --output_file='/content/rnn_model.tflite'  --keras_model_file='/content/rnn_cell.h5'
#   --graph_def_file='./gdrive/My Drive/model/tf_model_tempvsimple2.pb'


In [0]:
keras_file = 'linear.h5'
save_model(model, keras_file)

In [0]:
model.save('saved_model2')

W0614 17:19:26.636935 140454058903424 saved_model.py:722] Skipping full serialization of object <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x7fbde6dcfc10>, because an error occurred while tracing layer functions. Error message: wrapped_call() takes exactly 1 argument (3 given)
W0614 17:19:26.678813 140454058903424 saved_model.py:722] Skipping full serialization of object <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x7fbde5f53cd0>, because an error occurred while tracing layer functions. Error message: wrapped_call() takes exactly 1 argument (3 given)
W0614 17:19:26.720551 140454058903424 saved_model.py:722] Skipping full serialization of object <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fbdcd9a4550>, because an error occurred while tracing layer functions. Error message: wrapped_call() takes exactly 1 argument (3 given)


In [0]:
run_model = tf.function(lambda x : model(x))

# Save the concrete function.
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [0]:
model2 = tf.saved_model.load('/content/saved_model2')
concrete_func = model2.signatures[
  tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]

In [0]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()


In [0]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('linear.tflite', 'wb').write(tflite_model)

In [0]:
import tensorflow as tf

In [0]:
# Save the model.
# export_dir = "/tmp/test_saved_model"
# input_data = tf.constant(1., shape=[1, 1])
# to_save = root.f.get_concrete_function(input_data)
# tf.saved_model.save(model, export_dir)

# Convert the model.
converter = tf.lite.TocoConverter.fro (export_dir)
tflite_model = converter.convert()

In [0]:
!pip install tensorflow==1.9.0rc0

In [0]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
#import mnist dataset
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data/",one_hot=True)

#define constants
#unrolled through 28 time steps
time_steps=28
#hidden LSTM units
num_units=128
#rows of 28 pixels
n_input=28
#learning rate for adam
learning_rate=0.001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=10
#size of batch
batch_size=128
# os.makedirs('/content/saved')
output_dir='/content/saved'

In [0]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]),name="weights")
out_bias=tf.Variable(tf.random_normal([n_classes]),name="bias")

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1,name="input_tensor")

#defining the network
#lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
#lstm_layer=rnn.MultiRNNCell([rnn.BasicLSTMCell(num_units) for _ in range(3)])
#lstm_layer=rnn.LSTMBlockCell(num_units,forget_bias=1)
lstm_layer=tf.nn.rnn_cell.BasicLSTMCell(num_units)
#lstm_layer=tf.nn.rnn_cell.GRUCell(num_units)
#lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.add(tf.matmul(outputs[-1],out_weights), out_bias, name="output")

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    iter=1
    while iter<200:
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        # added
        saver = tf.train.Saver()
        filename = saver.save(sess, output_dir + '/model.ckpt')

        iter=iter+1

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

('For iter ', 10)
('Accuracy ', 0.3125)
('Loss ', 1.7808465)
__________________
('For iter ', 20)
('Accuracy ', 0.5390625)
('Loss ', 1.3754928)
__________________
('For iter ', 30)
('Accuracy ', 0.59375)
('Loss ', 1.227877)
__________________
('For iter ', 40)
('Accuracy ', 0.6875)
('Loss ', 1.0141048)
__________________
('For iter ', 50)
('Accuracy ', 0.7265625)
('Loss ', 0.8309381)
__________________
('For iter ', 60)
('Accuracy ', 0.7734375)
('Loss ', 0.72107434)
__________________
('For iter ', 70)
('Accuracy ', 0.8125)
('Loss ', 0.56332743)
__________________
('For iter ', 80)
('Accuracy ', 0.8046875)
('Loss ', 0.6424179)
__________________
('For iter ', 90)
('Accuracy ', 0.875)
('Loss ', 0.48204395)
__________________
('For iter ', 100)
('Accuracy ', 0.8515625)
('Loss ', 0.45273855)
________

In [0]:
import os, argparse

import tensorflow as tf

# The original freeze_graph function
# from tensorflow.python.tools.freeze_graph import freeze_graph

# dir = os.path.dirname(os.path.realpath(__file__))

def freeze_graph(model_dir, output_node_names):
    """Extract the sub graph defined by the output nodes and convert
    all its variables into constant
    Args:
        model_dir: the root folder containing the checkpoint state file
        output_node_names: a string, containing all the output node's names,
                            comma separated
    """
    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    if not output_node_names:
        print("You need to supply the name of a node to --output_node_names.")
        return -1

    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    # We precise the file fullname of our freezed graph
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"

    # We clear devices to allow TensorFlow to control on which device it will load operations
    clear_devices = True

    # We start a session using a temporary fresh Graph
    with tf.Session(graph=tf.Graph()) as sess:
        # We import the meta graph in the current default Graph
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

        # We restore the weights
        saver.restore(sess, input_checkpoint)

        # We use a built-in TF helper to export variables to constants
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess, # The session is used to retrieve the weights
            tf.get_default_graph().as_graph_def(), # The graph_def is used to retrieve the nodes
            output_node_names.split(",") # The output node names are used to select the usefull nodes
        )

        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

    return output_graph_def

In [0]:
freeze_graph('/content/saved','output')

INFO:tensorflow:Restoring parameters from /content/saved/model.ckpt
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.
507 ops in the final graph.


node {
  name: "weights"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 128
          }
          dim {
            size: 10
          }
        }
        tensor_content: "\032\217\262?\353-n\277\026\215\306?\t>=<\365(\033\300I\214<>\241\336-\277\311\341\311?t`\031\277j\233\266\276\3533\262?\314\n\231?3\177\262>\247\r\220>t \211\277\303\265\221\277\017\334\214=%i\307\277\213\\\270>v\2351>[\005E\276K\r\017?\344\231\305\277\323\037\010@\252\000\370?\374r\233>_\253\n@\372\354z?\257`\200\277{}y?\206F\364\276\302\271n\277\2323\221>\224\274\340\275\342\343\023?Q\327\215\277\253|\246\277/t\016@H\342\023?i\251\214=\254\335\235\277tx;<\302\255u>8\013\214\277\265\263C>\377_\311\277}2-?\311\237.?\236\335\261\276\217)\241\277KV\242\277\317X\360\277\210\212&\277\021;\240\277\327\377\345\277e^\001\300U\014\273\276h\322\032\277\330\3

In [0]:
!sudo apt-get install openjdk-8-jdk

In [0]:
!sudo add-apt-repository ppa:webupd8team/java

In [0]:
!sudo apt-get update && sudo apt-get install oracle-java8-installer

In [0]:
!echo "deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list

deb [arch=amd64] http://storage.googleapis.com/bazel-apt stable jdk1.8


In [0]:
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4654  100  4654    0     0  96958      0 --:--:-- --:--:-- --:--:-- 96958
OK


In [0]:
!sudo apt-get update && sudo apt-get install bazel

In [0]:
from tensorflow.python.tools import freeze_graph

In [0]:
output_node_name = "output"
restore_op_name = "saved/restore_all"
filename_tensor_name = "saved/Const:0"
clear_devices = True


# freeze_graph.freeze_graph(input_graph_path, input_saver_def_path, input_binary,
#                               checkpoint_path, output_node_name, restore_op_name,
#                               filename_tensor_name, output_frozen_graph_path,
#                               clear_devices, "")

# (directory, fn, ext) = splitDirFilenameExt(input_graph_path)
output_frozen_graph_path = '/content/rnn_frozen.pb'

freeze_graph.freeze_graph('/content/saved/frozen_model.pb', '/content/saved/model.ckpt.index', True,
                              '/content/saved/checkpoint', output_node_name, restore_op_name,
                              filename_tensor_name, output_frozen_graph_path,
                              clear_devices, "")

ValueError: ignored